In [2]:
import numpy as np, matplotlib, time, copy, random, math
#%load_ext line_profiler
from ortools.graph import pywrapgraph
directory = '../data/'
file_paths = ['a_example.txt', 'b_read_on.txt', 'c_incunabula.txt','d_tough_choices.txt', 'e_so_many_books.txt', 'f_libraries_of_the_world.txt']

### Classes

In [3]:
class Library():
    def __init__(self,index,N,T,M):
        self.id = index
        self.size  = N
        self.signup_time = T
        self.books_per_day = M
        self.book_ids = set()
        
    def add_book(self,book):
        self.book_ids.add(book)
        
    def remove_book(self,book):
        self.book_ids.remove(book)
        
    
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

### Utils

In [4]:
def process_file(filePath):
    with open(filePath,'r') as file:
        content = file.read().split('\n')[:-1];
        B,L,D = content[0].split()
        B,L,D = int(B),int(L),int(D)
    book_libraries = [set() for i in range(B)]
    bookValues = [int(n) for n in content[1].split()]
    libraries = []
    for i in range(L):
        N,T,M = content[2+2*i].split()
        N,T,M=int(N),int(T),int(M)
        book_ids = [int(id) for id in content[2+2*i+1].split()]
        library = Library(i,N,T,M)
        for book_id in book_ids:
            book_libraries[book_id].add(i)
            library.add_book(book_id)
        libraries.append(library)
    return ((B,L,D), bookValues,book_libraries, libraries)

In [5]:
def check_solution(D, libraries):
    days = 0
    prev_books = set()
    for library in libraries:
        days+=library.signup_time
        if len(library.book_ids) > (D-days)*library.books_per_day:
            print("what", len(library.book_ids), (D-days)*library.books_per_day)
        assert(len(library.book_ids) == len(set(library.book_ids)))
        assert(not any([(book in prev_books) for book in library.book_ids]))
        prev_books.update(library.book_ids)
    assert(days < D)
    
def score_solution(libraries, book_values):
    score=0
    for library in libraries:
        for book_id in library.book_ids:
            score+= book_values[book_id]
    return score

In [6]:
class ProblemSolver:
    def __init__(self, B, L, D, book_values, book_libraries, libraries):
        self.B, self.L, self.D = B,L,D
        self.book_values = book_values
        self.libraries = libraries
        self.book_libraries = copy.copy(book_libraries)
        
    def book_score(self, book):
        return self.book_values[book]
    
    def get_n_best_books(self,lib, n):
        return sorted(lib.book_ids,key=self.book_score, reverse=True)[:n]
        
    def get_solution(self,selected_lib_ids=None):
        if not selected_lib_ids:
            selected_lib_ids = self.get_individual()
        local_book_values = copy.copy(self.book_values)
        day = 0
        selected_libraries = [copy.copy(self.libraries[i]) for i in selected_lib_ids]
        it=0;
        already_scanned_books = set()
        while it < len(selected_libraries):
            next_library = selected_libraries[it]
            day+=next_library.signup_time
            if day>=self.D:
                break
            next_library.book_ids = next_library.book_ids - already_scanned_books
            next_library.book_ids = sorted(next_library.book_ids,key=self.book_score, reverse=True)[:(self.D-day)*next_library.books_per_day]
            already_scanned_books.update(next_library.book_ids)
            it+=1
        return selected_libraries[:it]
    
    def get_individual(self):
        pass

In [7]:
class HeurysticSolver(ProblemSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time
    
    def get_individual(self):
        lib_ids = [i for i in range(len(self.libraries))]
        lib_ids.sort(key = self.library_score, reverse= True)
        day = 0
        chosen = []
        for i in range(len(lib_ids)):
            if day + self.libraries[lib_ids[i]].signup_time >= self.D:
                continue  
            day+=self.libraries[lib_ids[i]].signup_time
            chosen.append(lib_ids[i])
        return tuple(chosen)

In [8]:
class PowerSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time**(1+lib.signup_time/self.D)

In [9]:
class SimpleScoreVarianceSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores/book_variance

In [10]:
class SquareScoreVarianceSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores**2/(lib.signup_time*lib.signup_time*math.sqrt(book_variance))

In [11]:
class BookNumbersSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        best_scores = [self.book_values[book] for book in n_best_books]
        sum_of_best_book_scores = sum(best_scores)
        book_variance = max(0.001,np.var(best_scores))
        return sum_of_best_book_scores**2/(book_variance*len(n_best_books)*lib.signup_time)

In [12]:
class ScoreSquareSolver(HeurysticSolver):
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores**2/lib.signup_time

In [13]:
class BookCountSolver(HeurysticSolver):
    def book_score(self, book):
        return self.book_values[book] - 0.7*len(self.book_libraries[book])

In [14]:
class BookCountPowerSolver(PowerSolver):
    def book_score(self, book):
        return self.book_values[book] - 0.7*len(self.book_libraries[book])

In [15]:
sum_score = 0
solvers_cls = [HeurysticSolver, PowerSolver, SimpleScoreVarianceSolver, SquareScoreVarianceSolver, BookNumbersSolver, ScoreSquareSolver, BookCountSolver, BookCountPowerSolver]
for file_path in file_paths:
    (B,L,D),book_values,book_counts, libraries = process_file(file_path)
    solvers = [cl(B,L,D,book_values,book_counts, libraries) for cl in solvers_cls]
    %time solutions = [solver.get_solution() for solver in solvers]
#     print(list(lib.id for lib in solutions[0]))
    [check_solution(D, solution) for solution in solutions]
    scores = [score_solution(solution, book_values) for solution in solutions]
    print(scores)
    score = max(scores)
    print(score)
    sum_score+=score
print("SUM",sum_score)

CPU times: user 412 µs, sys: 0 ns, total: 412 µs
Wall time: 416 µs
[21, 21, 21, 21, 21, 21, 21, 21]
21
CPU times: user 254 ms, sys: 3.43 ms, total: 258 ms
Wall time: 257 ms
[5822900, 5822900, 5822900, 5822900, 5822900, 5822900, 5822900, 5822900]
5822900
CPU times: user 1.16 s, sys: 755 µs, total: 1.16 s
Wall time: 1.16 s
[5645747, 5644690, 1191778, 5632921, 5309433, 5530843, 5645747, 5644690]
5645747
CPU times: user 2.87 s, sys: 30.6 ms, total: 2.9 s
Wall time: 2.86 s
[4815395, 4815395, 4815395, 4815395, 4815395, 4815395, 4815395, 4815395]
4815395
CPU times: user 1.1 s, sys: 0 ns, total: 1.1 s
Wall time: 1.1 s
[5020495, 5023616, 774717, 4791445, 2797585, 4741433, 5021339, 5024351]
5024351
CPU times: user 1.17 s, sys: 3.47 ms, total: 1.17 s
Wall time: 1.17 s
[5240161, 5238624, 1326571, 5211764, 4111186, 5083231, 5240157, 5238622]
5240161
SUM 26548575


In [16]:
class RandomSolver(ProblemSolver):
    def get_individual(self):
        lib_ids = [i for i in range(len(self.libraries))]
        random.shuffle(lib_ids)
        day = 0
        for i in range(len(lib_ids)):
            day += self.libraries[lib_ids[i]].signup_time
            if day >= self.D:
                return tuple(lib_ids[:i])
        return tuple(lib_ids)

In [17]:
class GreedyIntervalSolver:
    def __init__(self, B, L, D, book_values, book_libraries, libraries):
        self.B, self.L, self.D = B,L,D
        self.book_values = book_values
        self.libraries = libraries
        self.book_libraries = copy.copy(book_libraries)
        
    def book_score(self, book_id):
        return book_values[book_id] - 0.7*len(self.book_libraries[book_id])
    
    def get_n_best_books(self,lib, n):
        return sorted(lib.book_ids,key=self.book_score, reverse=True)[:n]
    
    def library_score(self,lib_id):
        lib = self.libraries[lib_id]
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores/lib.signup_time**(1+lib.signup_time/self.D)
    
    def get_solution(self):
        libraries = [copy.copy(library) for library in self.libraries]
        remaining_libraries = set(lib.id for lib in libraries)
        day = 0
        chosen_libraries = []
        it=0
        while len(remaining_libraries) > 0:
            if it%100 == 0:
                library_scores = [(self.library_score(lib_id),lib_id) for lib_id in remaining_libraries]
            it+=1
            max_el = max(library_scores, key = lambda x:x[0])
            library_scores.remove(max_el)
            score, lib_id = max_el
            remaining_libraries.remove(lib_id)
            library = libraries[lib_id]
            if day + library.signup_time>= self.D:
                break
            day+=library.signup_time
            books_to_take = (self.D - day)*library.books_per_day
            sorted_books = sorted(library.book_ids, key=self.book_score, reverse=True)
            for book_id in sorted_books[:books_to_take]:
                for lib_id in self.book_libraries[book_id]:
                    if lib_id != library.id:
                        libraries[lib_id].remove_book(book_id)

            for book_id in sorted_books[books_to_take:]:
                self.book_libraries[book_id].remove(library.id)

            library.book_ids = sorted_books[:books_to_take]
            chosen_libraries.append(library)
        return chosen_libraries
    
    def get_individual_from_solution(self,solution):
        return tuple([lib.id for lib in solution])

In [22]:
class GeneticSolver(ProblemSolver):
    def __init__(self, B, L, D, book_values,book_libraries, libraries, pop_size = 20, p_mutate = 0.5, surv_rate = 0.2, tournament_size = 3):
        super().__init__(B, L, D, book_values,book_libraries, libraries)
        num = 1
        mult = len(libraries)
        while num < pop_size and mult != 1:
            num*=mult
            mult-=1
        self.pop_size = min(pop_size,num)
        self.p_mutate = p_mutate
        self.tournament_size = min(self.pop_size,tournament_size)
        self.surv_rate = surv_rate
        
        self.indiv_scores = dict()
        self.lib_ids = [i for i in range(len(libraries))]
        self.lib_scores = [self.lib_score(lib) for lib in self.libraries]
        
    def lib_score(self, lib):
        delta_time = self.D - lib.signup_time
        n_best_books = self.get_n_best_books(lib, delta_time*lib.books_per_day)
        sum_of_best_book_scores = sum([self.book_values[book] for book in n_best_books])
        return sum_of_best_book_scores / lib.signup_time
        
        
    def mutate(self, individual):
        new_one = list(individual)
        if random.random() <=0.5:
            #internal mutation
            length = len(new_one)
            for i in range(4):
                a,b = random.sample(range(length), 2)
                new_one[a],new_one[b] =new_one[b],new_one[a]
            return tuple(individual)                  
        else:
            #exxternal mutation
            additional = random.choices(self.lib_ids, k=4, weights = self.lib_scores)
            unique = set(additional)
            for un in unique:
                new_one.insert(random.randrange(0,len(new_one)), un)
            return self.cut(new_one)
            
    def cut(self, new_one):
        day =0;
        for i in range(len(new_one)):
            day+=self.libraries[new_one[i]].signup_time
            if day >= self.D:
                return tuple(new_one[:i])
        return tuple(new_one)
    
    def crossover(self,indiv_1, indiv_2):
        set1 = set(indiv_1)
        set2 = set(indiv_2)
        common = set1.intersection(set2)
        maxLen = max(len(indiv_1), len(indiv_2))
        child1,child2,=[-1]*maxLen,[-1]*maxLen
        only1,only2 = [],[]
        for i in range(len(indiv_1)):
            if indiv_1[i] in common:
                child1[i] =  indiv_1[i]
            else:
                only1.append(indiv_1[i])
        for i in range(len(indiv_2)):
            if indiv_2[i] in common:
                child2[i] = indiv_2[i]
            else:
                only2.append(indiv_2[i])
        for i in range(maxLen):
            if child1[i] == -1 and len(only2) > 0:
                child1[i] = only2.pop(0)
            if child2[i] == -1 and len(only1) > 0:
                child2[i] = only1.pop(0)
        child1 = [i for i in child1 if i !=-1]
        child2 = [i for i in child2 if i !=-1]
                        
        child1 = self.cut(child1)
        child2 = self.cut(child2)
        if random.random()<=self.p_mutate:
            child1 = self.mutate(child1)
        if random.random()<=self.p_mutate:
            child2 = self.mutate(child2)
        return child1, child2
    
    def tournament(self, indivs):
        return max(indivs, key = self.indiv_scores.get);
    
    def select_survivals(self, population):
        k = int(5*self.pop_size*self.surv_rate)
        weights = [self.indiv_scores[indiv] for indiv in population]
        chosen = []
        already_chosen = set()
        desired = int(self.pop_size*self.surv_rate)
        while len(already_chosen) < desired:
            candidates = random.choices(population, k = k, weights = weights)
            for cand in candidates:
                if cand not in already_chosen:
                    chosen.append(cand)
                    already_chosen.add(cand)
        return chosen
    
    def get_initial_population(self):
        solvers_cls = [HeurysticSolver, PowerSolver, SimpleScoreVarianceSolver, SquareScoreVarianceSolver, BookNumbersSolver, ScoreSquareSolver, BookCountSolver, BookCountPowerSolver]
        solvers = [cl(self.B,self.L,self.D,self.book_values,self.book_libraries, self.libraries) for cl in solvers_cls]
        population = [solver.get_individual() for solver in solvers]
        greedyIntervalSolver = GreedyIntervalSolver(self.B,self.L,self.D,self.book_values,self.book_libraries, self.libraries)
        greedyIntervalSolution = greedyIntervalSolver.get_solution();
        gis_individual = greedyIntervalSolver.get_individual_from_solution(greedyIntervalSolution)
        population.append(gis_individual)
        self.indiv_scores[gis_individual] = score_solution(greedyIntervalSolution, self.book_values)
        randomSolver = RandomSolver(self.B,self.L,self.D,self.book_values,self.book_libraries, self.libraries)
        while len(population) < self.pop_size:
            population.append(randomSolver.get_individual())
        return population
    
    def get_individual(self):
        population = self.get_initial_population()
        for indiv in population:
            sol = self.get_solution(indiv)
            self.indiv_scores[indiv] = score_solution(sol, self.book_values)
        startTime = time.time()
        while(True):
            if(time.time() - startTime > 1):
                break
            new_population = [ self.tournament(random.sample(population, self.tournament_size)) for i in range(int(self.pop_size*self.surv_rate))]
            while len(new_population) < self.pop_size:
                indiv_1 = self.tournament(random.sample(population, self.tournament_size))
                indiv_2 = self.tournament(random.sample(population, self.tournament_size))
                if random.random()<= 0.5:
                    new_child1 = self.mutate(indiv_1)
                    new_child2 = self.mutate(indiv_2)
                else:
                    new_child1, new_child2 = self.crossover(indiv_1, indiv_2)
                if new_child1 not in self.indiv_scores:
                    self.indiv_scores[new_child1] = score_solution(self.get_solution(new_child1), book_values)
                if new_child2 not in self.indiv_scores:
                    self.indiv_scores[new_child2] = score_solution(self.get_solution(new_child2), book_values)
                new_population.append(new_child1)
                new_population.append(new_child2)
            population = tuple(new_population)
        return max(self.indiv_scores, key = self.indiv_scores.get)
              

In [19]:
sum_score =0
for file_path in file_paths[:2]:
    (B, L, D), book_values,book_counts, libraries = process_file(file_path)
    solver = GeneticSolver(B, L, D, book_values, book_counts, libraries, 20)
    solution = solver.get_solution()
    #print(solution)
    check_solution(D, solution)
    score = score_solution(solution, book_values)
    print("S: ",score)
    sum_score += score

print(sum_score)

S:  21
S:  5867300
5867321


In [20]:
sum_score =0
for file_path in file_paths:
    (B, L, D), book_values,book_counts, libraries = process_file(file_path)
    print(sum(book_values))

21
10000000
30076415
5109000
12548648
40111142


In [27]:
from copy import deepcopy
for file_path in file_paths[:2]:
    (B, L, D), book_values, book_counts, libraries = process_file(file_path)

    libraries2 = deepcopy(libraries)
    solver = GeneticSolver(B, L, D, book_values, book_counts, libraries, 20)
    solution = solver.get_individual()
    check_solution(D, solver.get_solution(selected_lib_ids=solution))
    score3 = score_solution(solver.get_solution(selected_lib_ids=solution), book_values)
    print(" xd:",score3)
    start_nodes = [0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 5, 6, 7, 8]
    end_nodes =   [1, 2, 3, 4, 5, 6, 7, 3, 5, 6, 8, 9, 9, 9, 9, 9, 9]
    capacities =  [10, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
    unit_costs =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -2, -3, -6, -5, -4]
    #print(solution)
    if len(solution) != len(set(solution)):
        print("shitasdsd")
    #solution = (38, 92, 27, 29, 48, 73, 77, 11, 34, 66, 88, 21, 32, 65, 75, 82, 18, 37, 57, 76, 2, 9, 22, 62, 14, 16, 31, 89, 13, 42, 46, 52, 53, 67, 69, 79, 84, 36, 40, 43, 55, 56, 61, 86, 90, 6, 30, 64, 78, 91, 59, 99, 7, 15, 58, 83, 3, 4, 12, 20, 28, 41, 45, 63, 81, 87, 5, 19, 23, 26, 85, 94, 44, 47, 50, 72, 74, 93, 8, 25, 33, 39, 70, 71, 80, 98, 54, 68, 0, 1)
    print(solution)
    # print(book_values)
    # for book in libraries2[1].book_ids:
    #     print(book)
    solution_len = len(solution)
    start_nodes1 = [0 for x in range(0, solution_len)]
    end_nodes1 = [x+1 for x in range(0, solution_len)]
    capacities1 = []
    unit_costs1 = [0 for x in range(0, solution_len)]
    #print(unit_costs1)


    logging_day = 0
    a_set = set()
    second_set = set()
    number_of_books = 0
    for library_index in solution:
        library = libraries2[library_index]
        #library.book_ids = sorted(library.book_ids)
        number_of_books += len(library.book_ids)
        how_many_books = (D-(logging_day + library.signup_time))*library.books_per_day
        logging_day += library.signup_time
        capacities1.append(how_many_books)

    for id_, library_index in enumerate(solution):
        library = libraries2[library_index]
        #library.book_ids = sorted(library.book_ids)
        for book in library.book_ids:
            a_set.add((book, book+solution_len+1))
            second_set.add((id_+1, book+solution_len+1))
            start_nodes1.append(id_+1)
            end_nodes1.append(book+solution_len+1)
            capacities1.append(1)
            unit_costs1.append(0)

    for book in a_set:
        start_nodes1.append(book[1])
        capacities1.append(1)
        unit_costs1.append(-book_values[book[0]])
    max_ =max(end_nodes1)+1
    sink = [max_ for x in range(0, len(a_set))]
    end_nodes1 += sink




    # print(len(start_nodes1))
    # print(len(end_nodes1))
    # print(len(capacities1))
    # print(len(unit_costs1))



    # print(start_nodes1)
    # print(end_nodes1)
    # print(capacities1)
    # print(unit_costs1)


    max_flow = pywrapgraph.SimpleMaxFlow()
  # Add each arc.
    for i in range(0, len(start_nodes1)):
        max_flow.AddArcWithCapacity(start_nodes1[i], end_nodes1[i], capacities1[i])
    max_flow.Solve(0, max_)
    optimal_flow = max_flow.OptimalFlow()
    print("optimal flow: ", optimal_flow)
    if max_flow.Solve(0, max_) != max_flow.OPTIMAL:
        raise Exception("shit")

    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

  # Add each arc.
    for i in range(0, len(start_nodes1)):
        min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes1[i], end_nodes1[i],
                                                capacities1[i], unit_costs1[i])

    # Add node supplies.
    supplies = [0 for x in range(0, max_-1)]
    supplies.insert(0, optimal_flow)
    supplies.append(-optimal_flow)
    #print(supplies)
    for i in range(0, len(supplies)):
        min_cost_flow.SetNodeSupply(i, supplies[i])
      # Find the minimum cost flow between node 0 and node 4.
    if min_cost_flow.Solve() == min_cost_flow.OPTIMAL:
        pass
        # print('Minimum cost:', min_cost_flow.OptimalCost())
        # print('')
        # print('  Arc    Flow / Capacity  Cost')
        # for i in range(min_cost_flow.NumArcs()):
        #     cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        #     print('%1s -> %1s   %3s  / %3s       %3s' % (
        #     min_cost_flow.Tail(i),
        #     min_cost_flow.Head(i),
        #     min_cost_flow.Flow(i),
        #     min_cost_flow.Capacity(i),
        #     cost))
    else:
        raise Exception("shit")
    new_solution =[]
    for x in solution:
        new_library = libraries2[x]
        new_library.book_ids = []
        new_solution.append(new_library)

    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        pair_1 = (min_cost_flow.Tail(i), min_cost_flow.Head(i))
        # print((min_cost_flow.Tail(i+number_of_books), min_cost_flow.Head(i+number_of_books)))
        if pair_1 in second_set and min_cost_flow.Flow(i) != 0 and min_cost_flow.Flow(i+number_of_books) !=0:
            new_solution[pair_1[0]-1].book_ids.append(pair_1[1]-solution_len-1)
        if i >= solution_len + number_of_books:
            break
#         print('%1s -> %1s   %3s  / %3s       %3s' % (
#         min_cost_flow.Tail(i),
#         min_cost_flow.Head(i),
#         min_cost_flow.Flow(i),
#         min_cost_flow.Capacity(i),
#         cost))

#     for solut in new_solution:
#         print(solut.book_ids)
    check_solution(D, new_solution)
    score = score_solution(new_solution, book_values)
    print("S: ",score)
    sum_score += score

 xd: 21
(0, 1)
optimal flow:  6
S:  21
 xd: 5840900
shitasdsd
(38, 92, 27, 29, 48, 73, 77, 11, 34, 66, 88, 21, 32, 65, 75, 82, 18, 37, 57, 92, 76, 2, 9, 22, 62, 14, 16, 31, 89, 13, 42, 46, 52, 53, 67, 38, 69, 79, 84, 36, 40, 43, 55, 56, 61, 86, 90, 6, 30, 64, 78, 91, 59, 99, 7, 15, 58, 83, 3, 4, 12, 20, 28, 41, 38, 45, 63, 81, 87, 5, 19, 23, 26, 7, 85, 94, 94, 61, 13, 44, 66, 47, 89, 50, 72, 48, 28, 29, 94, 50, 29, 74, 93, 8, 38, 69, 25, 33, 53, 16, 39, 40, 70)
optimal flow:  58409


AssertionError: 